In [0]:
from google.colab import files
uploaded = files.upload()


Saving test_2v.csv to test_2v.csv
Saving train_2v.csv to train_2v.csv


In [0]:
#from google.colab import drive
#drive.mount('/content/drive')


In [0]:
!ls

sample_data  test_2v.csv  train_2v.csv


In [0]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler,SMOTE, ADASYN
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve
from sklearn import svm
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix,auc,roc_auc_score,precision_score,recall_score

In [0]:
def model1(X_train,y_train,X_test,y_test):
    dtree = RandomForestClassifier(n_estimators=50,random_state=0)
    dtree.fit(X_train,y_train)

    pred = dtree.predict(X_test)
    #print(classification_report(y_test,pred)
    
    err = accuracy_score(y_test, pred)
    print(err)
    return err
    

In [0]:

train_data = pd.read_csv('train_2v.csv')
test_data = pd.read_csv('test_2v.csv')
train_data.shape
test_data.head()
print ('Train Data Shape: {}'.format(train_data.shape))

print ('Test Data Shape: {}'.format(test_data.shape))
train_data.describe()

train_data.isnull().sum()/len(train_data)*100
test_data.isnull().sum()/len(test_data)*100
joined_data = pd.concat([train_data,test_data])
print ('Joined Data Shape: {}'.format(joined_data.shape))
joined_data.isnull().sum()/len(joined_data)*100
train_data["bmi"]=train_data["bmi"].fillna(train_data["bmi"].mean())
train_data.head()
label = LabelEncoder()
train_data['gender'] = label.fit_transform(train_data['gender'])
train_data['ever_married'] = label.fit_transform(train_data['ever_married'])
train_data['work_type']= label.fit_transform(train_data['work_type'])
train_data['Residence_type']= label.fit_transform(train_data['Residence_type'])
train_data_without_smoke = train_data[train_data['smoking_status'].isnull()]
train_data_with_smoke = train_data[train_data['smoking_status'].notnull()]
train_data_without_smoke.drop(columns='smoking_status',axis=1,inplace=True)
train_data_without_smoke.head()
train_data_with_smoke.head()
train_data_with_smoke['smoking_status']= label.fit_transform(train_data_with_smoke['smoking_status'])
train_data_with_smoke.head()
train_data_with_smoke.shape
train_data_with_smoke.corr('pearson')
train_data_with_smoke['stroke'].value_counts()

train_data_without_smoke['stroke'].value_counts()
ros = RandomOverSampler(random_state=0)
smote = SMOTE()
X_resampled, y_resampled = ros.fit_resample(train_data_with_smoke.loc[:,train_data_with_smoke.columns!='stroke'], 
                                            train_data_with_smoke['stroke'])
train_data_without_smoke.loc[:,train_data_without_smoke.columns!='stroke'].columns
print ('ROS Input Data Shape for Smoke Data: {}'.format(X_resampled.shape))
print ('ROS Output Data Shape for Smoke Data: {}'.format(y_resampled.shape))
X_resampled_1, y_resampled_1 = ros.fit_resample(train_data_without_smoke.loc[:,train_data_without_smoke.columns!='stroke'], 
                                            train_data_without_smoke['stroke'])
print ('ROS Input Data Shape for Non Smoke Data: {}'.format(X_resampled_1.shape))
print ('ROS Output Data Shape for Non Smoke Data: {}'.format(y_resampled_1.shape))

#X_train,X_test,y_train,y_test = train_test_split(X_resampled,y_resampled,test_size=0.2)
#print(X_train.shape)
#print(X_test.shape)
#X_train_1,X_test_1,y_train_1,y_test_1 = train_test_split(X_resampled_1,y_resampled_1,test_size=0.2)
#print(X_train_1.shape)
#print(X_test_1.shape)





Train Data Shape: (43400, 12)
Test Data Shape: (18601, 11)
Joined Data Shape: (62001, 12)
ROS Input Data Shape for Smoke Data: (58940, 11)
ROS Output Data Shape for Smoke Data: (58940,)
ROS Input Data Shape for Non Smoke Data: (26294, 10)
ROS Output Data Shape for Non Smoke Data: (26294,)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

In [0]:
from numpy import array
from sklearn.model_selection import KFold
k=10
kfold = KFold(n_splits=k, random_state=102, shuffle=False)


total_error=0
for train, test in kfold.split(X_resampled):
    X_train=X_resampled[train]
    y_train=y_resampled[train]
    X_test=X_resampled[test]
    y_test=y_resampled[test]
    total_error+=model1(X_train,y_train,X_test,y_test)
    
kfold_error=total_error/k
print("kfold:",kfold_error)

0.997964031218188
0.9989820156090939
0.999830335934849
0.999491007804547
0.998642687478792
0.999830335934849
1.0
1.0
1.0
1.0
kfold: 0.9994740413980321
